In [1]:
import foolbox
from foolbox import Adversarial, attacks, models, criteria, distances
from simple_classifier import Classifier, get_data_loader
import torch
import argparse
from importlib import reload
from matplotlib import pyplot as plt
from advertorch.attacks import SpatialTransformAttack

In [2]:
device = torch.device('cpu')

In [3]:
reload(foolbox)

<module 'foolbox' from '/data/work2/AdversarialFaderNetworks/fader-torch/lib/python3.6/site-packages/foolbox/__init__.py'>

In [4]:
clf = Classifier((256,256,3))

In [5]:
clf.load_state_dict(torch.load('./new_class_model/best_model.pth', map_location='cpu'))
clf = clf.to(device)
clf.eval()

Classifier(
  (l1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (l2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (l3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=139392, out_features=1024, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=1024, out_features=2, bias=True)
  )
)

In [6]:
class Params():
    def __init__(self):
        self.attrib_path='/data/celebA/CelebA/Anno/corrected_list_attr.csv'
        self.data_dir='/data/celebA/CelebA/Img/img_align_celeba/'
        self.train_attribute='Male'
        self.batch_size=1

In [7]:
args2 = Params()

In [8]:
dl = get_data_loader(args2, train='test', shuffle=False)
acc = 0
for idx, (x, y) in enumerate(dl):
    x = x.to(device)
    y = y.to(device)
    y_pred = torch.argmax(clf(x))
    if y_pred == y:
        acc+=1
    if idx == 500:
        break

In [8]:
att = SpatialTransformAttack(clf, num_classes=2, search_steps=3, max_iterations=500)

In [ ]:
%mkdir xiao2

mkdir: cannot create directory ‘xiao2’: File exists


In [ ]:
%pwd

'/data/work2/attack_modules'

In [9]:
import numpy as np
success = 0
dl = get_data_loader(args2, train='test', shuffle=False)
for idx, (img, label) in enumerate(dl):
    print(idx)
    img = img.to(device)
    label = label.to(device)
    adv = att.perturb(img, label)
    if torch.argmax(clf(adv)).cpu().detach().numpy() != torch.argmax(clf(img)).cpu().detach().numpy():
        success += 1 
    #plt.imshow(img.detach().cpu().numpy()[0,...].transpose(1,2,0))
    #plt.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off', labelleft='off')
    #plt.savefig('xiao2/origimg_{}.png'.format(idx),bbox_inches='tight', pad_inches=0)
    #plt.figure()
    #plt.imshow(adv.detach().cpu().numpy()[0,...].transpose(1,2,0))
    #plt.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off', labelleft='off')
    #plt.savefig('xiao2/xiaoimg_{}.png'.format(idx),bbox_inches='tight', pad_inches=0)
    #plt.figure()
    #plt.imshow(np.hstack([img.detach().cpu().numpy()[0,...].transpose(1,2,0), adv.detach().cpu().numpy()[0,...].transpose(1,2,0)]))
    #print(idx, torch.argmax(clf(adv)), torch.argmax(clf(img)))
    if idx == 500:
        break
print(success)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
plt.imshow(img.detach().cpu().numpy()[0,...].transpose(1,2,0))
plt.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off', labelleft='off')
plt.savefig('origimg.png',bbox_inches='tight', pad_inches=0)

In [ ]:
plt.imshow(adv.detach().cpu().numpy()[0,...].transpose(1,2,0))
plt.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off', labelleft='off')
plt.savefig('xiaoimg.png',bbox_inches='tight', pad_inches=0)

In [ ]:
dl = get_data_loader(args2, train='test', shuffle=False)
criterion = criteria.Misclassification()
distance = distances.Linfinity
# adv_model = models.PyTorchModel(clf, (-10.0, 10.0), 2)
adv_model = models.PyTorchModel(clf, (-3.0, 3.0), 2)
# attack = attacks.CarliniWagnerL2Attack(adv_model, criterion, distance)
# attack = attacks.RandomProjectedGradientDescent(adv_model, criterion=criterion, distance=distance)
# attack = attacks.FGSM(adv_model, criterion=criterion, distance=distance)
attack = attacks.SpatialAttack(adv_model, criterion=criterion, distance=distance)
epses = []

In [ ]:
success = 0
for idx, (img,label) in enumerate(dl):
    img = img[0,...].numpy()
    label = label[0,...].numpy()
    # adv = attack(img, label, unpack=False, epsilon=1.64,binary_search=False,iterations=1000)
    adv = attack(img, label, unpack=False, angular_limits=(-5, 5))
    # adv = attack(img, label, unpack=False, max_epsilon=2.0)
    if adv.image is not None:
        # print(adv.image.shape)
 #       plt.imshow(adv.image.transpose(1,2,0))
        success +=1
        epses.append(adv.distance)
        print('{}:{}'.format(idx, epses[-1]))
        #print(adv.image)
    if idx == 500:
        break
#plt.show()

In [ ]:
print(success)

FGSM: 42
pGD: 42

In [ ]:
205/500.

In [ ]:
?distances.Linfinity

In [ ]:
adv_model.predictions_and_gradient(img, label)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(img.transpose(1,2,0))

In [ ]:
adv

In [ ]:
epses_pgd =epses

In [ ]:
import numpy as np

In [ ]:
epses_mod = [i.value for i in epses]

In [ ]:
np.mean(epses_mod)

In [ ]:
import pandas as pd

In [ ]:
np.save('cwl2.npy', epses_mod)

In [ ]:
epses_fgsm = [i.value for i in epses]

In [ ]:
epses_fgsm

In [ ]:
np.save('fgsm_k.npy', epses_fgsm)

In [ ]:
np.mean(epses_fgsm)

In [ ]:
epses_2 = [i for i in epses_fgsm if i!=0.0]

In [ ]:
np.mean(epses_2)

In [ ]:
1-42./500.

In [ ]:
np.ln(0.01)

In [ ]:
epses_pgd = [i.value for i in epses]

In [ ]:
np.mean(epses_pgd)

In [ ]:
len(epses)

In [ ]:
np.save('pgd.npy', epses_pgd)

In [ ]:
?attacks.SpatialAttack.__call__

In [ ]:
20/500 * 100

In [ ]:
epses_spat = [i.value for i in epses]

In [ ]:
np.mean(epses_spat)

In [ ]:
np.save('spat.npy', epses_spat)

In [ ]:
480-379

In [ ]:
101/500.

In [ ]:
500 - 325

In [ ]:
175/500.

In [ ]:
480 - 312

In [ ]:
168/480.

In [ ]:
500-369

In [ ]:
131/500

In [ ]:
1/np.sqrt(3000)

In [ ]:
500-336

In [ ]:
352/500.

In [ ]:
 1 - 298/500.

In [ ]:
%pwd

In [11]:
1 - 479/500.

0.04200000000000004